# Data RDM Creatation and Model Comparison Notebook

*More complicated script*

This script constructs a data RDM for each searchlight sphere. i.e. there is a `RDMs` object for each searchlight.

Steps for analysis. 

1. Get the correct directory for the analysis
2. Create the searchlight (composed of centers and neighbors)
   1. From the mask of the functional parts of the brain. The preprocess part of the brain that corresponds to doing computation. 
3. Load in the EVs that correspond to the correct parts of the BOLD video of the participant doing the task. 
   - i.e. The periods of time (in the BOLD video) that are part of the instruction period of the task.
   - Then load in the EVs, using nibabel, into np.arrays for each instruction period of the task for each half of the task 
4. Create the RDMs for each searchlight for the data of the participant. 
   This will return a object `data_RDMs` that contains the `RDMs` for each searchlight that was created in step 2. 
5. Compute the similarity between the `RDMs` of that we defined and compared it to every searchlight `RDMs` in the data.


OPTIONS: 
1. Subject number
2. EVs in question ("instruction_period")
3. RDM type ("replay_analysis")
4. Remove autocorrelation between task half 


In [32]:
# Automatically reload modules that have changed
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Imports 

In [33]:
# Python libraries
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sys
import random
import os
from pathlib import Path
from joblib import Parallel, delayed

# RSA specific libraries
import nibabel as nib
from nilearn.image import load_img
import rsatoolbox.rdm as rsr
import rsatoolbox.vis as vis
import rsatoolbox
from rsatoolbox.rdm import RDMs
from rsatoolbox.util.searchlight import get_volume_searchlight, get_searchlight_RDMs

# mc imports
import mc.analyse.analyse_MRI_behav     as analyse_MRI_behav
import mc.analyse.calc                  as calc
import mc.replay_analysis.functions.model_rdms as model_rdm_functions

### Options for analysis

In [41]:
# Constants
REGRESSION_VERSION = '01' 
RDM_VERSION        = '01-2' 
SUB_NO = '01'


USE_PREVIOUS_SEARCHLIGHTS = True  # Searchlights are loaded from file
USE_PREVIOUS_DATA_RDM     = False # Data RDMs are loaded from file
VISUALISE_RDMS            = False # Visualise the data RDMs
REMOVE_AUTOCORR           = True  # Remove autocorrelations from the data RDMs, else cross-correlate
EVS_TYPE                  = "instruction_period"     # The parts of the BOLD signal that are used for the RSA

# Paths
data_folder = Path('/Users/student/PycharmProjects/data')

# Subjects to be analysed
sub: str = f"sub-{SUB_NO}"
#subjects = ['sub-01']
task_halves: list = ['1', '2']

In [42]:
print(f"Now running RSA for RDM version {RDM_VERSION} based on subj GLM {REGRESSION_VERSION} for subj {SUB_NO}")

# get the list of the models to be analysed
models_I_want: list = analyse_MRI_behav.models_I_want(RDM_VERSION)

# based on GLM, get the number of conditions in the RDM
no_RDM_conditions: int = analyse_MRI_behav.get_no_RDM_conditions(RDM_VERSION)

Now running RSA for RDM version 01-2 based on subj GLM 01 for subj 01


## Get the correct directory for the analysis

In [43]:

subject_directory = data_folder / 'derivatives' / sub
print(subject_directory)
if os.path.isdir(subject_directory):
    print("Running on laptop.")


RDM_dir = subject_directory / 'beh' / f"RDMs_{RDM_VERSION}_glmbase_{REGRESSION_VERSION}"


# Make the RDM_dir if it doesn't exist
if not RDM_dir.exists():
    RDM_dir.mkdir(parents=True)
results_dir = subject_directory / 'func' / f"RSA_{RDM_VERSION}_glmbase_{REGRESSION_VERSION}"

# Make the results_dir if it doesn't exist
# if not results_dir.exists():
#     results_dir.mkdir(parents=True)
#     os.makedirs(f"{results_dir}/results")
# results_dir = f"{subject_directory}/func/RSA_{RDM_VERSION}_glmbase_{REGRESSION_VERSION}/results" 
# if os.path.exists(results_dir):
#     # move pre-existing files into a different folder.
#     analyse_MRI_behav.move_files_to_subfolder(results_dir)
# get a reference image to later project the results onto. This is usually
# example_func from half 1, as this is where the data is corrected to.

# results_dir = results_dir / 'results' 
ref_img = load_img( subject_directory / 'func' / 'preproc_clean_01.feat'/ 'example_func.nii.gz' )

# load the file which defines the order of the model RDMs, and hence the data RDMs
with open(f"{RDM_dir}/sorted_keys-model_RDMs.pkl", 'rb') as file:
    sorted_keys = pickle.load(file)
with open(f"{RDM_dir}/sorted_regs.pkl", 'rb') as file:
    reg_keys = pickle.load(file)
# also store 2 dictionaries of the EVs


/Users/student/PycharmProjects/data/derivatives/sub-01
Running on laptop.


In [37]:

REGRESSION_VERSION = analyse_MRI_behav.preprocess_regression_version(REGRESSION_VERSION)

# create dictionary of paths to the EVs for each half (split) of the task
# first half
EV_path_dict_01 = analyse_MRI_behav.get_EV_dict(
    subject_directory, REGRESSION_VERSION
    )
# second half
EV_path_dict_02 = analyse_MRI_behav.get_EV_dict(
    subject_directory, REGRESSION_VERSION
    )


## Get the search lights

In [38]:
# Step 1: get the searchlights

# Load the functional mask of the subject (i.e. the brain mask of only the computation parts of the brain)
mask = load_img(f"{subject_directory}/anat/{sub}_T1w_noCSF_brain_mask_bin_func_01.nii.gz")

# Get the searchlights
centers, neighbors = model_rdm_functions.get_searchlights(
    mask = mask,
    radius = 3, 
    threshold = 0.5,
    USE_PREVIOUS_SEARCHLIGHTS = USE_PREVIOUS_SEARCHLIGHTS,
    NEIGHBORS_PATH=f"{RDM_dir}/searchlight_neighbors.pkl",
    CENTERS_PATH=f"{RDM_dir}/searchlight_centers.pkl",
    SAVE_SEARCHLIGHTS = True
)


## Get the correct EVs from the BOLD video
(for the participant). This comes from a proprocessing step

In [39]:

# Step 2: loading and computing the data RDMs
if USE_PREVIOUS_DATA_RDM == True:
    # Open the data RDM file
    with open(f"{results_dir}/data_RDM.pkl", 'rb') as file:
        data_RDM = pickle.load(file)
    
    if VISUALISE_RDMS == True:
        analyse_MRI_behav.visualise_data_RDM(mni_x = 53, 
                                             mni_y = 30, 
                                             mni_z = 2, 
                                             data_RDM_file = data_RDM, 
                                             mask = mask)
        
else:
    # Create dictionary to store the data for each EV for both task halves
    EVs_both_halves_dict = {
        '1': None,
        '2': None
    }
    # create new dictionary to store the 2D array of EVs for both task halves
    EVs_both_halves_2d = EVs_both_halves_dict.copy()

    for split in task_halves:

        
        EVs_path_dict = model_rdm_functions.get_EV_path_dict(
            subject_directory = subject_directory,
            split = split,
            EVs_type = EVS_TYPE
            )
        
        # Load in the EVs for the instruction periods from the dictionary of paths
        EVs_data_dict = model_rdm_functions.load_EV_data(
            EVs_path_dict = EVs_path_dict,
            RDM_VERSION = RDM_VERSION
        )

        # Convert the dictionary of EVs to a 2D np.array (10 * 746496) (10 conditions, 746496 voxels)
        EVs_data_2d = model_rdm_functions.EV_data_dict_to_2d(
            EVs_data_dict = EVs_data_dict,
        )
        
        # Put the 2D array of EVs into the EV_both_halves dictionary
        EVs_both_halves_dict[split] = EVs_data_dict
        EVs_both_halves_2d  [split] = EVs_data_2d

        # Each part has array of shape (n_conditions, x, y, z)
        EVs_both_halves_dict[split] = np.array(list(EVs_both_halves_dict[split].values()))
        # Remove NaNs
        EVs_both_halves_dict[split] = np.nan_to_num(EVs_both_halves_dict[split])

        # Each part has array of shape (n_conditions, n_voxels)
        EVs_both_halves_2d[split] = EVs_both_halves_dict[split].reshape(EVs_both_halves_dict[split].shape[0], -1)

    # Combine the EVs from both task halves into a single 2D array (condition, x, y, z)
    EVs_both_halves_array_2d = np.concatenate((EVs_both_halves_2d['1'], EVs_both_halves_2d['2']), axis=0)

    # Remove NaNs
    # EVs_both_halves_array_2d = np.nan_to_num(EVs_both_halves_array_2d)

    # define the condition names for both task halves
    # 2 * 10 conditions, since there are 10 identical executution conditions in each task half
    # data_conds = np.reshape(np.tile((np.array(['cond_%02d' % x for x in np.arange(no_RDM_conditions)])), (1, 2)).transpose(), 2 * no_RDM_conditions)
    data_conds = [x for x in range(20)]
    # data_conds = np.reshape(np.tile((np.array(['cond_%02d' % x for x in np.arange(no_RDM_conditions)])), (1)).transpose(), no_RDM_conditions)

    # Defining both task halves runs: 0s first half, 1s is second half
    sessions = np.concatenate((np.zeros(int(EVs_both_halves_2d['1'].shape[0])),   # 10 of each condition
                                np.ones(int(EVs_both_halves_2d['2'].shape[0]))))  # 10 of each condition


## Create the RDMs for each searchlight in the 
*Takes a while*
This function returns a `data_RDMs` object that contains the RDM matrix for each searchlight (centers, neighbors) in the BOLD video

In [40]:



# for all other cases, cross correlated between task-halves.
# TODO: try to have one only half 
data_RDM = get_searchlight_RDMs(
    # data_2d = EVs_both_halves_2d,         # (nObs x nVox) (20 * 746496)
    data_2d = EVs_both_halves_array_2d, # (nObs x nVox)
    centers = centers, 
    neighbors = neighbors, 
    events  = data_conds,                 # (nObs x 1) of condition labels (con§d_00, cond_01, ... cond_09)
    method = 'correlation', 
    # method  ='crosscorr', 
    # cv_descr = sessions                   # (nObs x 1) of session labels (0, 1)
                )

# Save the data RDMs
with open(f"{results_dir}/data_RDM.pkl", 'wb') as file:
    pickle.dump(data_RDM, file)


Calculating RDMs...:  39%|███▉      | 39/100 [00:10<00:15,  3.95it/s]/Users/student/anaconda3/envs/rsa/lib/python3.11/site-packages/rsatoolbox/rdm/calc.py:217: RuntimeWarning: invalid value encountered in divide
  ma /= np.sqrt(np.einsum('ij,ij->i', ma, ma))[:, None]
Calculating RDMs...: 100%|██████████| 100/100 [00:26<00:00,  3.83it/s]


## Load in the Model RDMs object from previous script
For Alif's implementation, the Model RDMs will be already created from the "vector" models of the data that are created in previous model RDM script

In [16]:
# # Step 3: Load  Model RDMs, created in `replay_RSA_model_RDM.py`
# RDM_object_dict = {}

# RDM_object_dict['replay'] = model_rdm_functions.load_model_RDMs(
#     RDM_dir = RDM_dir,
    
#     )

# model_RDM_dict = {}
# Step 4: Run the RSA
# model_RDM_dict["replay"] = rsr.calc_rdm(
#     dataset = replay_RDM_object, 
#     method='correlation', 
#     descriptor='conds'
#     )


replay_dir = "/Users/student/PycharmProjects/data/derivatives/sub-01/beh/RDMs_01_glmbase_01/replay_RDM_object.pkl"

with open(replay_dir, 'rb') as file:
    replay_RDM_object = pickle.load(file)

# data_RDM = model_rdm_functions.load_model_RDMs(replay_dir)
# print("Running RSA")

In [17]:

# # Step 4: Create Fixed Model RDMs
# def prepare_model_RDM_dict(
#     model_RDMs: dict,
#     no_RDM_conditions: int,
#     RDM_VERSION: str,
#     MODEL_TYPE: str = "matrix",
#     VISUALISE_RDMS: bool = False,
#     REMOVE_AUTOCORR: bool = True
# ):
#     """
#     Parameters
#         neuron_model_RDMs: dictionary of model RDMs
#         no_RDM_conditions: number of conditions in the RDM
#         RDM_VERSION: version of the RDM
#         MODEL_TYPE: "neuron" represent the vector form of the model RDM. "matrix" represents the matrix form of the model RDM
#         REMOVE_AUTOCORR: remove autocorrelations from the model RDMs. If True it takes into account to the two task halves

#     Returns
#         model_RDM_dir: dict Dictionary of model RDMs
#     """

#     # Where, each model gets its own, separate estimation.
#     model_RDM_dir = {}

#     if MODEL_TYPE == "neuron":
#         # Use the vectors for the different conditions to create the RDMs

#         for model in model_RDMs:
#             # 4.1 either prepare the neuron_modlel and place it into the `analyse_MRI_behav.prepare_model_data function`, then run the `rsr.calc_rdm` function
#             # prepare the neuron_model of the data into the Dataset object that is used in rsr.calc_rdm()
#             model_data = analyse_MRI_behav.prepare_model_data(
#                 model_RDMs[model],
#                 no_RDM_conditions,
#                 RDM_VERSION)
            
#             if REMOVE_AUTOCORR == False:
#                 # assumes that both task halves are one session
#                 model_RDM_dir[model] = rsr.calc_rdm(
#                     model_data, 
#                     method='correlation', 
#                     descriptor='conds'
#                     )

#             else:
#                 # assumes that there are two task halves, that need to be corrected for autocorrelations
#                 model_RDM_dir[model] = rsr.calc_rdm(
#                     model_data, 
#                     method='crosscorr', 
#                     descriptor='conds', 
#                     cv_descriptor='sessions'
#                     )

#     elif MODEL_TYPE == "matrix":
#         # takes a dictionary of matrix RDMs as the correctly packaged into the RDMs object in a dictionary

#         for model in model_RDMs:
            
#             # get the model from the model_RDMs dictionary
#             model_RDM = model_RDMs[model]

#             if REMOVE_AUTOCORR == False:
#                 model_RDM_dir[model] = rsr.calc_rdm(
#                     dataset = model_RDM,
#                     method = 'correlation',
#                     descriptor = 'conds'
#                 )
            
#             else: 
#                 model_RDM_dir[model] = rsr.calc_rdm(
#                     dataset = model_RDM,
#                     method = 'crosscorr',
#                     descriptor = 'conds',
#                     cv_descriptor = 'sessions'
#                 )

#     if VISUALISE_RDMS == True:
#         # Visualise the model RDMs
#         fig, ax, ret_vla = rsatoolbox.vis.show_rdm(model_RDM_dir[model])


#     return model_RDM_dir

# # Prepare the model RDMs
# model_RDM_dict = prepare_model_RDM_dict(
#     model_RDMs = neuron_model_RDMs,
#     no_RDM_conditions = no_RDM_conditions,
#     RDM_VERSION = RDM_VERSION,
#     MODEL_TYPE = "neuron",
#     VISUALISE_RDMS = VISUALISE_RDMS,
#     REMOVE_AUTOCORR = REMOVE_AUTOCORR
# )


# def run_fixed_model(
#     model_RDM_dir: dict
# ):
#     """

#     # Step 4.2: evaluate the model fit between model and data RDMs.
#     """
#     # Dictionary to store the results
#     RDM_my_model_dir = {}

#     for model in model_RDM_dir:
#         # Define the type of model to be evaluated. It is a single model, not a set of models that. It does not have a set of betas to also fit. 
#         single_model = rsatoolbox.model.ModelFixed(f"{model}_only", model_RDM_dir[model])
#         # Run the model evaluation for all searchlights
#         RDM_my_model_dir[model] = Parallel(n_jobs=3)(delayed(analyse_MRI_behav.evaluate_model)(single_model, data_RDM_p_voxel) for data_RDM_p_voxel in tqdm(data_RDM, desc=f"running GLM for all searchlights in {model}"))
        

#     # return dictionary of results
#     return RDM_my_model_dir


# # Run the fixed model
# RDM_my_model_dir = run_fixed_model(
#     model_RDM_dir = model_RDM_dict
# )


In [18]:
import statsmodels.api as sm

replay_dir = "/Users/student/PycharmProjects/data/derivatives/sub-01/beh/RDMs_01_glmbase_01/replay_RDM_object.pkl"
with open(replay_dir, 'rb') as file:
    replay_RDM_object = pickle.load(file)


model_RDM_object = analyse_MRI_behav.prepare_model_data(
    model_data = replay_RDM_object,
    no_Conditions = no_RDM_conditions,
    RDM_version= "01"
)



AttributeError: 'RDMs' object has no attribute 'transpose'

### Note that the single_model.__dict__ is a readout of the model RDM object. This should match formatting of the data RDMs object. (`data_RDM_p_voxel`). 
These are the things being compared so they ahve to match. 

In [83]:
# replay_RDM_object
single_model.__dict__

{'name': 'replay_only',
 'n_param': 0,
 'default_fitter': <function rsatoolbox.model.fitter.fit_mock(model, data, method='cosine', pattern_idx=None, pattern_descriptor=None, sigma_k=None)>,
 'rdm_obj': rsatoolbox.rdm.RDMs(
 dissimilarity_measure = 
 Arbitrary
 dissimilarities = 
 [[-1.    0.25  0.    0.25  0.    0.25  0.    0.25  0.    0.    1.   -0.25
    0.   -0.25  0.   -0.25  0.   -0.25  0.    0.    0.25  0.    0.25  0.
    0.25  0.    0.25  1.    0.    0.   -0.25  0.   -0.25  0.   -0.25  0.
   -0.25 -1.    0.25  0.    0.25  0.    0.25  0.   -0.25  0.    0.    1.
   -0.25  0.   -0.25  0.   -0.25  0.    0.    0.25  0.    0.25  0.    0.25
    0.   -0.25  1.    0.    0.   -0.25  0.   -0.25  0.   -0.25 -1.    0.25
    0.    0.25  0.   -0.25  0.   -0.25  0.    0.    1.   -0.25  0.   -0.25
    0.    0.    0.25  0.    0.25  0.   -0.25  0.   -0.25  1.    0.    0.
   -0.25  0.   -0.25 -1.    0.25  0.   -0.25  0.   -0.25  0.   -0.25  0.
    0.    1.   -0.25  0.    0.    0.25  0.   -0.25  0. 

In [86]:
# single_model.rdm.shape, replay_RDM_object.dissimilarities.shape 
data_RDM_p_voxel

rsatoolbox.rdm.RDMs(
dissimilarity_measure = 
crosscorr
dissimilarities = 
[[1.18280327 1.03554441 1.16859723 0.83408911 1.18755462 1.19309875
  1.06420735 1.14512869 1.09228604 0.35876889 0.42071846 0.83255953
  0.49742841 0.91828871 0.35357708 0.48662118 0.78141249 1.3330743
  1.42478664 1.29528497 1.13690648 1.28888624 1.06374565 1.15682004
  0.78523534 0.34180407 0.95716977 0.386003   0.32168538 0.73678456
  1.14594754 1.19582381 1.17198995 1.00953275 0.98233452 1.301615
  1.3696185  1.38305538 0.95839203 0.95554143 0.67282555 1.00517554
  0.48016354 0.68537069 1.14973713]]
descriptors = 
{}
rdm_descriptors = 
{'voxel_index': [162003], 'index': [0]}
pattern_descriptors = 
{'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}

# Evaluate the model RDMs with the data RDMs
For the evaluation function to work, we need to the model and data RDMs need to be in the same format.

In [19]:
# Dictionary to store the results
RDM_my_model_dir = {}
# Define the type of model to be evaluated. It is a single model, not a set of models that. It does not have a set of betas to also fit. 
model = "replay"
single_model = rsatoolbox.model.ModelFixed(f"{model}_only", replay_RDM_object)

# Run the model evaluation for all searchlights
RDM_my_model_dir[model] = Parallel(n_jobs=3)(delayed(analyse_MRI_behav.evaluate_model)(single_model, data_RDM_p_voxel) for data_RDM_p_voxel in tqdm(data_RDM, desc=f"running GLM for all searchlights in {model}"))


running GLM for all searchlights in replay: 100%|██████████| 172178/172178 [00:09<00:00, 18636.21it/s]


In [31]:
# type(RDM_my_model_dir[model][0][0])
RDM_my_model_dir[model][0][0].shape

(1,)

In [20]:

# Step 4.3: Save the results
for model in RDM_my_model_dir:
    # Save the different aspects of the model as different nii files 
    analyse_MRI_behav.save_RSA_result(
                        result_file = RDM_my_model_dir[model], 
                        data_RDM_file = data_RDM, 
                        file_path = results_dir, 
                        file_name = f"{model}", 
                        mask = mask, 
                        number_regr = 0, 
                        ref_image_for_affine_path=ref_img
                        )
        
